In [1]:
def loaddataMT5(symbol, from_date, to_date):
    import MetaTrader5 as mt5
    from datetime import datetime
    import pandas as pd 

    # Kết nối tới MetaTrader 5
    if not mt5.initialize():
        print("Khởi tạo MT5 không thành công")
        mt5.shutdown()

    from_date_str = datetime.strptime(from_date, '%Y-%m-%d')
    to_date_str = datetime.strptime(to_date, '%Y-%m-%d')
    
    # Lấy dữ liệu OHLC cho cặp tiền symbol trong khoảng thời gian đã xác định
    ohlc_data = mt5.copy_rates_range(symbol, mt5.TIMEFRAME_D1, from_date_str, to_date_str)
    # Ngắt kết nối với MT5
    mt5.shutdown()

    # Chuyển dữ liệu nhận được thành DataFrame và hiển thị
    data = pd.DataFrame(ohlc_data)
    data['time'] = pd.to_datetime(data['time'], unit='s')

    # ohlc_df.reset_index(inplace=True)

    data = data.rename(columns={'time': 'Datetime'})        
    data = data.rename(columns={'open': 'Open'})       
    data = data.rename(columns={'high': 'High'})       
    data = data.rename(columns={'low': 'Low'})       
    data = data.rename(columns={'close': 'Close'})       
    data = data.rename(columns={'tick_volume': 'Volume'})       

    data = pd.DataFrame(data, columns=['Datetime', 'Open', 'High', 'Low', 'Close', 'Volume'])

    return data

In [2]:
def loaddataMT5_FromTo(symbol, from_date, to_date, timeframe):
    import MetaTrader5 as mt5
    from datetime import datetime
    import pandas as pd 

    # Kết nối tới MetaTrader 5
    if not mt5.initialize():
        print("Khởi tạo MT5 không thành công")
        mt5.shutdown()

    from_date_str = datetime.strptime(from_date, '%Y-%m-%d')
    to_date_str = datetime.strptime(to_date, '%Y-%m-%d')
    
    # Lấy dữ liệu OHLC cho cặp tiền symbol trong khoảng thời gian đã xác định
    ohlc_data = mt5.copy_rates_range(symbol, timeframe, from_date_str, to_date_str)
    # Ngắt kết nối với MT5
    mt5.shutdown()

    # Chuyển dữ liệu nhận được thành DataFrame và hiển thị
    data = pd.DataFrame(ohlc_data)
    data['time'] = pd.to_datetime(data['time'], unit='s')

    # ohlc_df.reset_index(inplace=True)

    data = data.rename(columns={'time': 'Datetime'})        
    data = data.rename(columns={'open': 'Open'})       
    data = data.rename(columns={'high': 'High'})       
    data = data.rename(columns={'low': 'Low'})       
    data = data.rename(columns={'close': 'Close'})       
    data = data.rename(columns={'tick_volume': 'Volume'})       

    data = pd.DataFrame(data, columns=['Datetime', 'Open', 'High', 'Low', 'Close', 'Volume'])

    return data

In [3]:
import MetaTrader5 as mt5
from datetime import datetime, timedelta
import schedule
import time
import pandas as pd
import plotly.graph_objects as go
import numpy as np

# ##############################################Step 1: Lấy dữ liệu##############################################
symbol = 'USDJPY'
from_date = (datetime.now() - timedelta(days=5)).strftime('%Y-%m-%d')  # Lấy ngày hôm qua
to_date = (datetime.now() + timedelta(days=1)).strftime('%Y-%m-%d')
timeframe = 1 # 1m # 0x4000
data = loaddataMT5_FromTo(symbol, from_date, to_date, timeframe)

data

,Datetime,Open,High,Low,Close,Volume
0,2024-01-24 17:00:00,147.060,147.073,146.977,147.029,218
1,2024-01-24 17:01:00,147.027,147.057,146.982,147.029,224
2,2024-01-24 17:02:00,147.031,147.077,147.023,147.069,253
3,2024-01-24 17:03:00,147.067,147.179,147.067,147.177,269
4,2024-01-24 17:04:00,147.176,147.207,147.127,147.128,219
...,...,...,...,...,...,...
2692,2024-01-30 07:03:00,147.357,147.367,147.354,147.367,28
2693,2024-01-30 07:04:00,147.367,147.377,147.364,147.376,17
2694,2024-01-30 07:05:00,147.376,147.376,147.348,147.353,18
2695,2024-01-30 07:06:00,147.354,147.357,147.352,147.353,9


# Lay 1 danh sach

In [4]:
# Import necessary modules
from ssi_fc_data import fc_md_client, model
import pandas as pd
import json
import configDataSSL

# Create a Market Data Client
client = fc_md_client.MarketDataClient(configDataSSL)

listStock = ['VCB', 'ACB', 'TCB', 'TPB']  # Changed to a list for ordered processing

# Initialize an empty list for all stock data
all_stock_data = []

# Iterate over each stock in the list
for stock in listStock:
    req = model.daily_ohlc(stock, "05/09/2023", "06/09/2023")
    data_dict = client.daily_ohlc(configDataSSL, req)
    
    if 'data' in data_dict:
        data_list = data_dict['data']
        df = pd.DataFrame(data_list)
        all_stock_data.append(df)
    else:
        print(f"No data returned for {stock}")

# Concatenate all DataFrames in the list into a single DataFrame
all_stock_data_df = pd.concat(all_stock_data, ignore_index=True)

# Print or work with the all_stock_data_df DataFrame
print(all_stock_data_df)


  Symbol Market TradingDate  Time   Open   High    Low  Close    Volume  \
0    VCB   HOSE  05/09/2023  None  89100  90700  89100  90100   1942200   
1    VCB   HOSE  06/09/2023  None  90200  90200  89700  90200   2017600   
2    ACB   HOSE  05/09/2023  None  22700  23050  22650  22750   7626800   
3    ACB   HOSE  06/09/2023  None  22750  22950  22600  22950   4778100   
4    TCB   HOSE  05/09/2023  None  34800  35150  34750  35000   7998900   
5    TCB   HOSE  06/09/2023  None  35050  35350  34850  35200   4653200   
6    TPB   HOSE  05/09/2023  None  19700  19850  19600  19650   9227200   
7    TPB   HOSE  06/09/2023  None  19650  19750  19150  19750  11948100   

               Value  
0       174726890000  
1  181566640000.0010  
2  174315814999.9990  
3       108877055000  
4  279663229999.9990  
5  163120430000.0010  
6  182003740000.0020  
7  232676664999.9970  


# Lay 1 danh sach Forex

In [5]:
def loaddataMT5_FromTo_List(symbol, from_date, to_date, timeframe):
    import MetaTrader5 as mt5
    from datetime import datetime
    import pandas as pd 

    # Kết nối tới MetaTrader 5
    if not mt5.initialize():
        print("Khởi tạo MT5 không thành công")
        mt5.shutdown()

    from_date_str = datetime.strptime(from_date, '%Y-%m-%d')
    to_date_str = datetime.strptime(to_date, '%Y-%m-%d')
    
    # Lấy dữ liệu OHLC cho cặp tiền symbol trong khoảng thời gian đã xác định
    ohlc_data = mt5.copy_rates_range(symbol, timeframe, from_date_str, to_date_str)
    # Ngắt kết nối với MT5
    mt5.shutdown()

    # Chuyển dữ liệu nhận được thành DataFrame và hiển thị
    data = pd.DataFrame(ohlc_data)
    data['time'] = pd.to_datetime(data['time'], unit='s')

    # ohlc_df.reset_index(inplace=True)
    data['Symbol'] = symbol
    data = data.rename(columns={'time': 'Datetime'})        
    data = data.rename(columns={'open': 'Open'})       
    data = data.rename(columns={'high': 'High'})       
    data = data.rename(columns={'low': 'Low'})       
    data = data.rename(columns={'close': 'Close'})       
    data = data.rename(columns={'tick_volume': 'Volume'})       

    data = pd.DataFrame(data, columns=['Symbol', 'Datetime', 'Open', 'High', 'Low', 'Close', 'Volume'])

    return data

In [6]:
import MetaTrader5 as mt5
from datetime import datetime, timedelta
import schedule
import time
import pandas as pd
import plotly.graph_objects as go
import numpy as np

# ##############################################Step 1: Lấy dữ liệu##############################################
listsymbol = ['USDJPY', 'EURUSD']
all_symbol_data = []

# Fetch data for each symbol and append to the list
for symbol in listsymbol:
    from_date = (datetime.now() - timedelta(days=5)).strftime('%Y-%m-%d')  # Lấy ngày hôm qua
    to_date = (datetime.now() + timedelta(days=1)).strftime('%Y-%m-%d')
    timeframe = 1  # 1 minute timeframe
    data = loaddataMT5_FromTo_List(symbol, from_date, to_date, timeframe)
    all_symbol_data.append(data)  # Append the data to the list

# Concatenate all data into a single DataFrame
all_stock_data_df = pd.concat(all_symbol_data, ignore_index=True)

print(all_stock_data_df)  # Display the concatenated data

       Symbol            Datetime       Open       High        Low      Close  \
0      USDJPY 2024-01-24 17:00:00  147.06000  147.07300  146.97700  147.02900   
1      USDJPY 2024-01-24 17:01:00  147.02700  147.05700  146.98200  147.02900   
2      USDJPY 2024-01-24 17:02:00  147.03100  147.07700  147.02300  147.06900   
3      USDJPY 2024-01-24 17:03:00  147.06700  147.17900  147.06700  147.17700   
4      USDJPY 2024-01-24 17:04:00  147.17600  147.20700  147.12700  147.12800   
...       ...                 ...        ...        ...        ...        ...   
10208  EURUSD 2024-01-30 07:03:00    1.08239    1.08239    1.08228    1.08229   
10209  EURUSD 2024-01-30 07:04:00    1.08226    1.08234    1.08224    1.08233   
10210  EURUSD 2024-01-30 07:05:00    1.08233    1.08237    1.08229    1.08233   
10211  EURUSD 2024-01-30 07:06:00    1.08233    1.08234    1.08228    1.08231   
10212  EURUSD 2024-01-30 07:07:00    1.08231    1.08237    1.08230    1.08235   

       Volume  
0         2